In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.cluster import DBSCAN,AgglomerativeClustering,KMeans
from sklearn.mixture import GaussianMixture
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_excel("Cuestionario Diagnóstico 2019 - 1 (Respuestas).xlsx")
df.head(5)

,Marca temporal,Carrera,Género,Edad 🎂,Estado civil,¿Tienes hijos?,¿Tienes alguna dificultad para? [👓 Ver],¿Tienes alguna dificultad para? [👂 Escuchar],¿Tienes alguna dificultad para? [💪 Realizar alguna actividad física],¿Dónde vives actualmente?,...,¿Qué tanto sabes sobre...? [Los maestros de la carrera],¿Qué tanto sabes sobre...? [El cuerpo directivo de la carrera y la universidad],¿Qué tanto sabes sobre...? [Actividades deportivas y culturales que ofrece la facultad],¿Qué tanto sabes sobre...? [Recursos de apoyo ],¿Cuántos días a la semana dedicas tiempo para realizar las siguientes actividades? [Deportes ⚽️🏀🏈],¿Cuántos días a la semana dedicas tiempo para realizar las siguientes actividades? [Actividades culturales 🎭🎬🎻],"¿Cuántos días a la semana dedicas tiempo para realizar las siguientes actividades? [Actividades sociales (fiestas, reuniones con amigos, familia, etc.)]","¿Cuántos días a la semana dedicas tiempo para realizar las siguientes actividades? [Entretenimiento personal (videojuegos, redes sociales, netflix, youtube, etc)]",Al terminar la carrera ¿qué esperas hacer además de titularte?,Número de cuenta
0,2018-10-17 16:46:19.603,Matemáticas Aplicadas y Computación,Hombre,18,👤 Soltero,No,Si,No,No,CDMX,...,Regular,Poco,Regular,Regular,0,0,0,2,Formar mi propia empresa,NaN
1,2018-10-17 16:52:37.461,Matemáticas Aplicadas y Computación,Mujer,18,💍 Casado,Sí,Si,No,No,Estado de México,...,Poco,Regular,Regular,Regular,0,0,1,2,Trabajar en una empresa,NaN
2,2018-10-17 16:54:48.531,Matemáticas Aplicadas y Computación,Hombre,18,👤 Soltero,No,No,No,No,Estado de México,...,Regular,Poco,Poco,Nada,0,0,0,7,Formar mi propia empresa,NaN
3,2018-10-17 17:07:29.619,Matemáticas Aplicadas y Computación,Hombre,42,👤 Soltero,No,No,Si,No,Estado de México,...,Regular,Regular,Poco,Regular,1,0,1,2,Formar mi propia empresa,NaN
4,2018-10-17 17:26:57.906,Matemáticas Aplicadas y Computación,Hombre,18,👥 En una relación,No,Si,No,No,CDMX,...,Regular,Regular,Regular,Regular,0,1,2,5,Trabajar en una empresa y luego formar una mía.,NaN


In [3]:
def Proc_freq_sin_aumulados(data,var):
    
    tabla_aux = pd.DataFrame(data)
    
    table_freq = pd.value_counts(tabla_aux[var]).to_frame().reset_index()
    table_freq.columns = [var,'Frequency']

    table_porc = pd.value_counts(tabla_aux[var],normalize = True).to_frame().reset_index()
    table_porc.columns = [var,'Percent']
    
    table_salida = pd.merge(table_freq, table_porc, on=var)
    table_salida.sort_values([var],axis = 0, ascending=True, inplace=True) 
       
    return table_salida

In [4]:
print(df['Carrera'].dtype)

object


In [5]:
tipo = df['Edad 🎂'].dtype
print(tipo)

int64


In [6]:
def mi_desc(data):
    dic = []
    for var_i in data.columns:
        Uniq_i = data[var_i].nunique()
        
    
        if Uniq_i <= 10: Tipo_var_i = 'Discreta'
        else : Tipo_var_i = 'Continua'
        Vacios_i = data[var_i].isnull().sum()
        
        tipo_var_i = data[var_i].dtype
        if tipo_var_i == 'object' : Tipo_var_i = 'Discreta'
        
        h = dict(Variable = var_i,
                     Unicos = Uniq_i,
                     Tipo_var = Tipo_var_i,
                     Vacios = Vacios_i
                    )
        dic.append(h) 
    dic
    data2 = pd.DataFrame(dic)
    return data2

In [7]:
tabla_desc_1 = mi_desc(df)
tabla_desc_1.sort_values(['Tipo_var','Unicos'],axis = 0, ascending=True, inplace=True) 

tabla_desc_1

,Tipo_var,Unicos,Vacios,Variable
3,Continua,11,0,Edad 🎂
15,Continua,12,0,¿Cuántas personas en total viven en tu hogar? ...
32,Continua,14,104,¿Cuántas horas trabajas a la semana?
18,Continua,25,0,¿Cuánto tiempo llevas viviendo ahí? (años)
11,Continua,32,0,¿Cuánto tiempo dura (en minutos) el trayecto d...
51,Continua,54,0,¿A cuánto aproximadamente ascienden los ingres...
38,Continua,78,0,¿Qué promedio obtuviste en el bachillerato?
0,Continua,228,0,Marca temporal
80,Discreta,1,227,Número de cuenta
1,Discreta,2,0,Carrera


In [8]:
Proc_freq_sin_aumulados(tabla_desc_1,'Tipo_var')

,Tipo_var,Frequency,Percent
1,Continua,8,0.098765
0,Discreta,73,0.901235


In [9]:
list_Cont = tabla_desc_1[(tabla_desc_1["Tipo_var"] == "Continua")].filter(items = ['Variable'])
list_Cont.sort_values(['Variable'], axis = 0, ascending=True, inplace=True) 
list_Cont.set_index('Variable', inplace=True)

list_Cont


""
Variable
Edad 🎂
Marca temporal
¿A cuánto aproximadamente ascienden los ingresos de tu hogar(considerando a todos los que aportan)?
¿Cuántas horas trabajas a la semana?
¿Cuántas personas en total viven en tu hogar? Incluyéndote a ti
¿Cuánto tiempo dura (en minutos) el trayecto de tu casa a la facultad? ⏱
¿Cuánto tiempo llevas viviendo ahí? (años)
¿Qué promedio obtuviste en el bachillerato?


In [10]:
list_Cont = list_Cont.T
list_Cont

Variable,Edad 🎂,Marca temporal,¿A cuánto aproximadamente ascienden los ingresos de tu hogar(considerando a todos los que aportan)?,¿Cuántas horas trabajas a la semana?,¿Cuántas personas en total viven en tu hogar? Incluyéndote a ti,¿Cuánto tiempo dura (en minutos) el trayecto de tu casa a la facultad? ⏱,¿Cuánto tiempo llevas viviendo ahí? (años),¿Qué promedio obtuviste en el bachillerato?


In [11]:
ls_var = list_Cont.columns
ls_var

Index(['Edad 🎂', 'Marca temporal',
       '¿A cuánto aproximadamente ascienden los ingresos de tu hogar(considerando a todos los que aportan)?',
       '¿Cuántas horas trabajas a la semana?',
       '¿Cuántas personas en total viven en tu hogar? Incluyéndote a ti',
       '¿Cuánto tiempo dura (en minutos) el trayecto de tu casa a la facultad? ⏱',
       '¿Cuánto tiempo llevas viviendo ahí? (años)',
       '¿Qué promedio obtuviste en el bachillerato?'],
      dtype='object', name='Variable')

In [12]:
df_cont = df[ls_var]
df_cont.head(5)

,Edad 🎂,Marca temporal,¿A cuánto aproximadamente ascienden los ingresos de tu hogar(considerando a todos los que aportan)?,¿Cuántas horas trabajas a la semana?,¿Cuántas personas en total viven en tu hogar? Incluyéndote a ti,¿Cuánto tiempo dura (en minutos) el trayecto de tu casa a la facultad? ⏱,¿Cuánto tiempo llevas viviendo ahí? (años),¿Qué promedio obtuviste en el bachillerato?
0,18,2018-10-17 16:46:19.603,4000,0.0,6,120.0,5,7.43
1,18,2018-10-17 16:52:37.461,5000,NaN,6,60.0,0,7.80
2,18,2018-10-17 16:54:48.531,8000,NaN,3,130.0,5,8.34
3,42,2018-10-17 17:07:29.619,13000,8.0,8,2.0,3,7.45
4,18,2018-10-17 17:26:57.906,35000,0.0,4,40.0,18,8.50


In [13]:
columnas = df_cont.columns
aux = 1
lst_var_old_names = []
lst_var_new_names = []
for nom_i in columnas:
    New_name = 'Var_' + f'{aux}'
    df_cont = df_cont.rename(columns={nom_i : New_name})
    print(nom_i ,'----', New_name)
    
    lst_var_old_names.append(nom_i)
    lst_var_new_names.append(New_name)
    
    aux = aux+1

df_cont.head(5)

Edad 🎂 ---- Var_1
Marca temporal ---- Var_2
¿A cuánto aproximadamente ascienden los ingresos de tu hogar(considerando a todos los que aportan)? ---- Var_3
¿Cuántas horas trabajas a la semana? ---- Var_4
¿Cuántas personas en total viven en tu hogar? Incluyéndote a ti ---- Var_5
¿Cuánto tiempo dura (en minutos) el trayecto de tu casa a la facultad? ⏱ ---- Var_6
¿Cuánto tiempo llevas viviendo ahí? (años) ---- Var_7
¿Qué promedio obtuviste en el bachillerato? ---- Var_8


,Var_1,Var_2,Var_3,Var_4,Var_5,Var_6,Var_7,Var_8
0,18,2018-10-17 16:46:19.603,4000,0.0,6,120.0,5,7.43
1,18,2018-10-17 16:52:37.461,5000,NaN,6,60.0,0,7.80
2,18,2018-10-17 16:54:48.531,8000,NaN,3,130.0,5,8.34
3,42,2018-10-17 17:07:29.619,13000,8.0,8,2.0,3,7.45
4,18,2018-10-17 17:26:57.906,35000,0.0,4,40.0,18,8.50
